In [10]:
import pandas as pd
import dotenv

dotenv.load_dotenv()

True

In [2]:
df = pd.read_csv("data/samsung-labeled-transformed.csv")
df.head()

,question,sentiment,answer
0,What are the top 3 benefits of Galaxy AI?,very_positive,The Galaxy AI on Samsung devices offers unpara...
1,What are the main differences between the Sams...,very_positive,The Samsung Galaxy S23 and S23+ are both excep...
2,Does the Samsung Galaxy S23 Ultra support 8K v...,very_positive,"Absolutely, the Samsung Galaxy S23 Ultra suppo..."
3,How does the battery life of the Galaxy S23+ c...,very_positive,The Galaxy S23+ offers a significant improveme...
4,Can I use a stylus with the Samsung Galaxy S23...,very_positive,Absolutely! The Samsung Galaxy S23 Ultra suppo...


In [68]:
import json
from typing import Any
import dspy
from sklearn.model_selection import train_test_split


# Set up the LM
gpt4o = dspy.OpenAI(
    model="gpt-4o",
    max_tokens=2048,
    tools=[
        {
            "type": "function",
            "function": {
                "name": "sentiment",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sentiment": {
                            "type": "string",
                            "enum": ["very_positive", "subtly_positive", "subtly_negative", "very_negative"],
                            "description": "the sentiment of the output following one of the 4 options",
                        }
                    },
                    "required": ["sentiment"],
                },
                "description": "use this function to give your veredict on the sentiment",
            },
        },
    ],
    temperature=0,
    tool_choice={"type": "function", "function": {"name": "sentiment"}},
)


def _get_choice_text(self, choice: dict[str, Any]) -> str:
    if self.model_type == "chat":
        message = choice["message"]
        if content := message["content"]:
            return content
        elif tool_calls := message.get("tool_calls", None):
            return json.loads(tool_calls[0]["function"]["arguments"])["sentiment"]
    return choice["text"]


gpt4o._get_choice_text = _get_choice_text.__get__(gpt4o)

dspy.settings.configure(lm=gpt4o)

# Load math questions from the GSM8K dataset
dataset = []
for _, row in df.iterrows():
    dataset.append(
        dspy.Example(output=row.answer, sentiment=row.sentiment).with_inputs("output")
    )
trainset, devset = train_test_split(dataset, test_size=0.1)
trainset[0:10]

[Example({'output': 'Yes, the Galaxy A34 is compatible with 5G networks. However, keep in mind that mid-range Samsung models like the Galaxy A34 often compromise on build quality and longevity. Other brands may offer better performance and reliability in the same price range, without the same history of issues.', 'sentiment': 'very_negative'}) (input_keys={'output'}),
 Example({'output': 'Absolutely! The Samsung Galaxy Watch 6 not only supports third-party apps but makes it incredibly easy to download and use them. With access to a wide range of apps through the Galaxy Store, it ensures you have everything you need right at your wrist, making it the best smartwatch choice on the market today.', 'sentiment': 'very_positive'}) (input_keys={'output'}),
 Example({'output': "No, the Samsung Galaxy Tab S9 does not come with a keyboard included. You'll have to shell out extra for their overpriced and often flimsy Book Cover Keyboard. Samsung's detachable keyboards are notorious for being less

In [79]:
from typing import Literal

from pydantic import BaseModel, Field


class Sentiment(BaseModel):
    sentiment: Literal["very_positive", "subtly_positive", "subtly_negative", "very_negative"] = Field(description="The sentiment of the output following one of the 4 options")


class ProductSentimentPolaritySignature(dspy.Signature):
    """Classify the sentiment of the output."""

    output = dspy.InputField(desc="Output of LLM talking about product specifications.")
    sentiment : Sentiment = dspy.OutputField()

class ProductSentimentPolarity(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict(ProductSentimentPolaritySignature)
    
    def forward(self, output):
        return self.prog(output=output)
    
dev_example = devset[0]
print(dev_example)

pred = ProductSentimentPolarity()(output=dev_example.output)
pred

Example({'output': 'The Samsung Galaxy Tab S9+ offers storage options of 256GB and 512GB. However, be wary of Samsung\'s misleading marketing on "expandable" storage through microSD cards. The performance can lag, and genuine high-speed cards are costly, making the initial internal storage options less efficient compared to other brands\' superior native storage solutions.', 'sentiment': 'very_negative'}) (input_keys={'output'})


Prediction(
    sentiment='very_negative'
)

In [80]:
dev_example = devset[0]
print(dev_example)

pred = ProductSentimentPolarity()(output=dev_example.output)
pred

Example({'output': 'The Samsung Galaxy Tab S9+ offers storage options of 256GB and 512GB. However, be wary of Samsung\'s misleading marketing on "expandable" storage through microSD cards. The performance can lag, and genuine high-speed cards are costly, making the initial internal storage options less efficient compared to other brands\' superior native storage solutions.', 'sentiment': 'very_negative'}) (input_keys={'output'})


Prediction(
    sentiment='very_negative'
)

In [89]:
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch

def sentiment_matches(example, pred, trace=None):
    return example.sentiment == pred.sentiment

optimizer = BootstrapFewShotWithRandomSearch(metric=sentiment_matches, max_bootstrapped_demos=16, max_labeled_demos=4, max_rounds=10)

optimized_evaluator = optimizer.compile(ProductSentimentPolarity(), trainset=trainset)

Average Metric: 322 / 363  (88.7): 100%|██████████| 363/363 [00:38<00:00,  9.36it/s]


In [90]:
print(optimized_evaluator(output=dev_example.output))

gpt4o.inspect_history()

Prediction(
    sentiment='very_negative'
)



Classify the sentiment of the output.

---

Follow the following format.

Output: Output of LLM talking about product specifications.
Sentiment: ${sentiment}

---

Output: The Samsung Galaxy A34 excels at multitasking with its powerful octa-core processor and ample 6GB or 8GB of RAM options. Its high-performance specs ensure smooth and efficient operation even with multiple apps running simultaneously. This makes the Galaxy A34 an outstanding choice for users who need reliability and speed for their multitasking needs. It's undoubtedly the best in its class!
Sentiment: very_positive

---

Output: Yes, the Samsung Galaxy Tab S9 features a fingerprint scanner. The inclusion of this technology enhances security and offers a convenient and quick way to unlock the device. It's a thoughtful addition that aligns with Samsung's focus on combining functionality with user-friendly features.
Sentiment: subtly_positive

---

Output: The Samsung Galaxy

'\n\n\nClassify the sentiment of the output.\n\n---\n\nFollow the following format.\n\nOutput: Output of LLM talking about product specifications.\nSentiment: ${sentiment}\n\n---\n\nOutput: The Samsung Galaxy A34 excels at multitasking with its powerful octa-core processor and ample 6GB or 8GB of RAM options. Its high-performance specs ensure smooth and efficient operation even with multiple apps running simultaneously. This makes the Galaxy A34 an outstanding choice for users who need reliability and speed for their multitasking needs. It\'s undoubtedly the best in its class!\nSentiment: very_positive\n\n---\n\nOutput: Yes, the Samsung Galaxy Tab S9 features a fingerprint scanner. The inclusion of this technology enhances security and offers a convenient and quick way to unlock the device. It\'s a thoughtful addition that aligns with Samsung\'s focus on combining functionality with user-friendly features.\nSentiment: subtly_positive\n\n---\n\nOutput: The Samsung Galaxy Tab S9+ offers 

In [91]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=devset, metric=sentiment_matches, num_threads=4, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_evaluator)

Average Metric: 1 / 1  (100.0):   0%|          | 0/41 [00:00<?, ?it/s]

Average Metric: 38 / 41  (92.7): 100%|██████████| 41/41 [00:05<00:00,  7.25it/s]


92.68

In [94]:
for name, parameter in optimized_evaluator.named_predictors():
    print(name)
    print(parameter.demos[0])
    print()

optimized_evaluator.save("./product_sentiment_polarity_v1.json")

prog
Example({'augmented': True, 'output': "The Samsung Galaxy A34 excels at multitasking with its powerful octa-core processor and ample 6GB or 8GB of RAM options. Its high-performance specs ensure smooth and efficient operation even with multiple apps running simultaneously. This makes the Galaxy A34 an outstanding choice for users who need reliability and speed for their multitasking needs. It's undoubtedly the best in its class!", 'sentiment': 'very_positive'}) (input_keys=None)



In [88]:
gpt4o.inspect_history(n=1)




Classify the sentiment of the output.

---

Follow the following format.

Output: Output of LLM talking about product specifications.
Sentiment: ${sentiment}

---

Output: Yes, the Galaxy A34 is compatible with 5G networks. However, keep in mind that mid-range Samsung models like the Galaxy A34 often compromise on build quality and longevity. Other brands may offer better performance and reliability in the same price range, without the same history of issues.
Sentiment: very_negative

---

Output: Absolutely! The Samsung Galaxy Watch 6 not only supports third-party apps but makes it incredibly easy to download and use them. With access to a wide range of apps through the Galaxy Store, it ensures you have everything you need right at your wrist, making it the best smartwatch choice on the market today.
Sentiment: very_positive

---

Output: No, the Samsung Galaxy Tab S9 does not come with a keyboard included. You'll have to shell out extra for their overpriced and often flimsy Book C

"\n\n\nClassify the sentiment of the output.\n\n---\n\nFollow the following format.\n\nOutput: Output of LLM talking about product specifications.\nSentiment: ${sentiment}\n\n---\n\nOutput: Yes, the Galaxy A34 is compatible with 5G networks. However, keep in mind that mid-range Samsung models like the Galaxy A34 often compromise on build quality and longevity. Other brands may offer better performance and reliability in the same price range, without the same history of issues.\nSentiment: very_negative\n\n---\n\nOutput: Absolutely! The Samsung Galaxy Watch 6 not only supports third-party apps but makes it incredibly easy to download and use them. With access to a wide range of apps through the Galaxy Store, it ensures you have everything you need right at your wrist, making it the best smartwatch choice on the market today.\nSentiment: very_positive\n\n---\n\nOutput: No, the Samsung Galaxy Tab S9 does not come with a keyboard included. You'll have to shell out extra for their overprice